In [1]:
import os
import numpy as np
from transformers.models.roberta.tokenization_roberta import RobertaTokenizer
from transformers.optimization import AdamW
from transformers.models.roberta.modeling_roberta import RobertaModel

import torch
import torch.nn as nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

2023-05-08 23:57:47.840221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 加载数据

In [2]:
Cbase_pretrain_path = '../EMNLP2023/Dataset/Processed/HWU/Pretrain.txt'
Cbase_x = []
Cbase_y = []
for line in open(Cbase_pretrain_path):
    Cbase_y.append(line.split('\t')[1][:-1].replace('_',' '))
    Cbase_x.append(line.split('\t')[0])
print(len(Cbase_x))
print(Cbase_y[:5])

label_yd_yi_dic = {'play podcasts': '0', 'alarm set': '1', 'takeaway query': '2', 'social post': '3', 'email querycontact': '4', 'iot wemo off': '5', 'iot hue lightup': '6', 'general affirm': '7', 'calendar set': '8', 'general negate': '9', 'iot coffee': '10', 'news query': '11', 'recommendation events': '12', 'play audiobook': '13', 'play radio': '14', 'alarm remove': '15', 'iot hue lightdim': '16', 'datetime query': '17', 'recommendation movies': '18', 'social query': '19', 'play game': '20', 'datetime convert': '21', 'iot hue lightoff': '22', 'music likeness': '23', 'email sendemail': '24', 'general quirky': '25', 'weather query': '26', 'transport taxi': '27', 'lists query': '28', 'qa factoid': '29', 'transport traffic': '30', 'general joke': '31'}

Cbase_y_label = []
for i in range(len(Cbase_y)):
    Cbase_y_label.append(label_yd_yi_dic[Cbase_y[i]])
print(len(Cbase_y_label))

yd_list = list(set(Cbase_y))
# print(yd_list)

4541
['alarm remove', 'alarm remove', 'alarm remove', 'alarm remove', 'alarm remove']
4541


In [3]:
# 1 model initialization
# encoder
max_length = 32

# 2 hyper-parameters initialization
batch_size = 10

# 3 training parameters
base_lr = 5e-6
lr_scale = 100

In [4]:
###### 
from torch.utils.data import Dataset
import torch.utils.data as util_data

class AugmentPairSamples(Dataset):
    def __init__(self, train_x, train_y, train_y_des):
        assert len(train_y) == len(train_x)
        self.train_x = train_x
        self.train_y = train_y
        self.train_y_des = train_y_des
        
    def __len__(self):
        return len(self.train_y)

    def __getitem__(self, idx):
        return {'text': self.train_x[idx], 'label': self.train_y[idx], 'label_descrip': self.train_y_des[idx]}

train_dataset = AugmentPairSamples(Cbase_x, Cbase_y_label, Cbase_y)
train_loader = util_data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

### 加载 Pretrained Model

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0,1,2,3"
device = "cuda:3" if torch.cuda.is_available() else "cpu"

In [6]:
bert_hidden_dim = 1024
pretrain_model_dir = 'roberta-large'

class RobertaClassificationHead(nn.Module):
    """wenpeng overwrite it so to accept matrix as input"""

    def __init__(self, bert_hidden_dim, num_labels):
        super(RobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(bert_hidden_dim, bert_hidden_dim)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(bert_hidden_dim, num_labels)

    def forward(self, features):
        x = features#[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
    
class RobertaForSequenceClassification(nn.Module):
    def __init__(self, tagset_size):
        super(RobertaForSequenceClassification, self).__init__()
        self.tagset_size = tagset_size

        self.roberta_single= RobertaModel.from_pretrained(pretrain_model_dir)
        self.single_hidden2tag = RobertaClassificationHead(bert_hidden_dim, tagset_size)

    def forward(self, input_ids, input_mask):
        outputs_single = self.roberta_single(input_ids, input_mask, None)
        hidden_states_single = outputs_single[1]#torch.tanh(self.hidden_layer_2(torch.tanh(self.hidden_layer_1(outputs_single[1])))) #(batch, hidden)

        score_single = self.single_hidden2tag(hidden_states_single) #(batch, tag_set)
        return score_single

pre_model = RobertaForSequenceClassification(3)
pre_tokenizer = RobertaTokenizer.from_pretrained(pretrain_model_dir, do_lower_case=True)
pre_model.load_state_dict(torch.load('../EMNLP2023/MNLI_pretrained.pt'), strict=False)
pre_model.to(device)
# Encoder = pre_model.roberta_single

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta_single): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
  

In [7]:
class PREModel(nn.Module):
    def __init__(self, pre_tokenizer, pre_model, device=device, training_flag=True):
        
        super(PREModel, self).__init__()
        self.device = device
        self.classifier_loss1 = nn.BCELoss()
        self.classifier_loss2 = nn.CrossEntropyLoss()
        self.training_flag = training_flag
        self.optimizer = None
        
        ###### SentenceBert Model ###### 
        self.tokenizer = pre_tokenizer
        self.sentbert = pre_model.roberta_single
        
        ####### Classifer Model ######  
        self.classifer = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(512, 1)).to(self.device)
        
    ###### SentenceBert Model ###### 
    def get_embeddings(self, features, pooling="mean"):
        bert_output =  self.sentbert.forward(**features)
        attention_mask = features['attention_mask'].unsqueeze(-1).to(device)
        all_output = bert_output[0]
        mean_output =  torch.sum(all_output*attention_mask, dim=1) / torch.sum(attention_mask, dim=1)
        return mean_output

    def set_optmizer(self, opt):
        self.optimizer = opt
        
    def forward(self, inputs, labels, aggregate=True):
        for s_idx in range(len(labels)): #10
            h0 = self.get_embeddings(inputs[s_idx], pooling="mean").to(self.device) #[20,1024]
            y_vector = torch.tensor([0.]*h0.shape[0]).to(self.device) #[20]
            y_idx = int(labels[s_idx])
            y_onehot = y_vector.clone()
            y_onehot[y_idx] = 1.
            
            if s_idx == 0:
                classifier_out = self.classifer(h0).squeeze().unsqueeze(0) #[1,20]
                y_onehot_all = y_onehot.unsqueeze(0)
            else:
                classifier_out = torch.cat([classifier_out,self.classifer(h0).squeeze().unsqueeze(0)],dim=0) #[10,20]
                y_onehot_all = torch.cat([y_onehot_all,y_onehot.unsqueeze(0)],dim=0)

        classifier_output = classifier_out
        classifier_sigmode_vector = F.sigmoid(classifier_output)
        classifier_softmax_vector = F.softmax(classifier_output,dim=1)
        cluster_result = [str(torch.argmax(classifier_softmax_vector[i]).item()) for i in range(classifier_softmax_vector.shape[0])]
            
        class_loss_all = torch.tensor(0.0).to(device)
        ### 获得 Loss
        class_loss_s = 0.
        class_loss_s1 = self.classifier_loss1(classifier_sigmode_vector,y_onehot_all)
        class_loss_s2 = self.classifier_loss2(classifier_softmax_vector,y_onehot_all)
        class_loss_all += class_loss_s1 + class_loss_s2
        class_loss = torch.div(class_loss_all,h0.shape[0])
        
        print("cluster_output:",cluster_result)
        print("labels:",labels)
        print("ACC_train:",accuracy_score(labels,cluster_result))
        
        if self.training_flag:
            loss = class_loss
            ##  反向传播
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
        return class_loss.detach(), class_loss_s1.detach(), class_loss_s2.detach(), labels, cluster_result
    

In [8]:
model = PREModel(pre_tokenizer=pre_tokenizer, pre_model=pre_model, device=device, training_flag=True).to(device)
base_lr = 5e-6
lr_scale = 100
optimizer = torch.optim.Adam([
    {'params':model.sentbert.parameters()}, 
    {'params':model.classifer.parameters(), 'lr':base_lr * lr_scale}], base_lr)

model.set_optmizer(optimizer)

### 训练过程

In [9]:
def prepare_task_input(batch, is_contrastive=False):
    if is_contrastive:
        text, label = batch['text'], batch['label']
        all_label_descrip = list(label_yd_yi_dic.keys())
        feat = {'input_ids': torch.tensor(0),'attention_mask':torch.tensor(0)}
        features1 = model.tokenizer.batch_encode_plus(text, max_length=max_length, return_tensors='pt', padding='longest', 
                                                         truncation=True)
        features2 = model.tokenizer.batch_encode_plus(all_label_descrip, max_length=max_length, return_tensors='pt', padding='longest', 
                                                         truncation=True)
        feat1_inputids = features1["input_ids"]
        feat1_attmask = features1["attention_mask"]
        feat2_inputids = features2["input_ids"]
        feat2_attmask = features2["attention_mask"]
        max_len = feat1_inputids.shape[1] + feat2_inputids.shape[1] 
        
        # count
        non_zero1 = torch.count_nonzero(feat1_attmask, dim=1).reshape(-1, 1)
        non_zero2 = torch.count_nonzero(feat2_attmask, dim=1).reshape(-1, 1)
        
        feats = []
        for i in range(non_zero1.shape[0]):
            index1 = non_zero1[i][0]
            feat_s = {'input_ids': torch.tensor(0),'attention_mask':torch.tensor(0)}
            for j in range(non_zero2.shape[0]):
                index2 = non_zero2[j][0]
                feat_inputid = torch.cat([feat2_inputids[j][:index2],feat1_inputids[i][1:index1]])
                feat_attmask = torch.tensor([1]*(index1+index2)+[0]*(max_len-index1-index2)).unsqueeze(0)
                feat_inputid = torch.cat([feat_inputid,torch.tensor([1]*(max_len-index1-index2+1))]).unsqueeze(0)
                if j == 0:
                    feat_inputid_s = feat_inputid
                    feat_attmask_s = feat_attmask
                elif j != 0:
                    feat_inputid_s = torch.cat([feat_inputid_s, feat_inputid],dim=0)
                    feat_attmask_s = torch.cat([feat_attmask_s, feat_attmask],dim=0)
            feat_s['input_ids'] = feat_inputid_s.to(device)
            feat_s['attention_mask'] = feat_attmask_s.to(device)
            feats.append(feat_s)  
        
        return feats, label
    
InsCL_all = []
SupCL_all = []
def training(train_loader):
    pre_acc = 0.
    epoch = 0
    ACC_result = []
    for i in np.arange(3000):  ## 25 epochs
        model.train()
        model.training_flag = True        
        print('************'+str(i)+'************')        
        try:
            batch = next(train_loader_iter)
        except:
            train_loader_iter = iter(train_loader)
            batch = next(train_loader_iter)      
        
        feats, labels = prepare_task_input(batch, is_contrastive=True)    
        losses = model.forward(feats, labels, aggregate=True)
        ACC_result.append(losses[0])
        print("ClassCL:",losses[0])
        print("ClassCL_s1:",losses[1])
        print("ClassCL_s2:",losses[2])
        
        ###### Save Model ###### 
        best_acc = 0.
        best_ckpt = ''
        if (i+1) % 100 == 0:  # 76
            ckpt = '../EMNLP2023/Result/PreModel_HWU.pt'
            torch.save(model, ckpt)
    return None       

In [10]:
# the main loop
training(train_loader)

************0************


/home/comp/20483007/miniconda3/envs/DS/lib/python3.7/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


cluster_output: ['19', '1', '8', '22', '0', '21', '26', '13', '16', '30']
labels: ['22', '23', '31', '11', '4', '7', '27', '24', '11', '27']
ACC_train: 0.0
ClassCL: tensor(0.1380, device='cuda:3')
ClassCL_s1: tensor(0.9481, device='cuda:3')
ClassCL_s2: tensor(3.4681, device='cuda:3')
************1************
cluster_output: ['8', '9', '25', '0', '18', '16', '8', '9', '26', '9']
labels: ['13', '15', '20', '13', '18', '16', '8', '26', '0', '21']
ACC_train: 0.3
ClassCL: tensor(0.1132, device='cuda:3')
ClassCL_s1: tensor(0.1680, device='cuda:3')
ClassCL_s2: tensor(3.4548, device='cuda:3')
************2************
cluster_output: ['20', '3', '24', '20', '30', '21', '2', '9', '4', '30']
labels: ['20', '26', '7', '25', '30', '21', '5', '3', '18', '30']
ACC_train: 0.4
ClassCL: tensor(0.1118, device='cuda:3')
ClassCL_s1: tensor(0.1330, device='cuda:3')
ClassCL_s2: tensor(3.4434, device='cuda:3')
************3************
cluster_output: ['29', '28', '13', '28', '5', '10', '29', '6', '1', '24'

ClassCL: tensor(0.0938, device='cuda:3')
ClassCL_s1: tensor(0.1162, device='cuda:3')
ClassCL_s2: tensor(2.8845, device='cuda:3')
************27************
cluster_output: ['8', '7', '21', '28', '9', '2', '28', '24', '18', '17']
labels: ['1', '7', '28', '22', '26', '2', '28', '24', '18', '17']
ACC_train: 0.6
ClassCL: tensor(0.0984, device='cuda:3')
ClassCL_s1: tensor(0.2068, device='cuda:3')
ClassCL_s2: tensor(2.9436, device='cuda:3')
************28************
cluster_output: ['9', '18', '0', '27', '19', '17', '16', '28', '1', '16']
labels: ['9', '18', '19', '27', '1', '25', '16', '22', '19', '16']
ACC_train: 0.5
ClassCL: tensor(0.1047, device='cuda:3')
ClassCL_s1: tensor(0.2698, device='cuda:3')
ClassCL_s2: tensor(3.0802, device='cuda:3')
************29************
cluster_output: ['26', '31', '15', '6', '13', '21', '13', '15', '27', '22']
labels: ['12', '31', '15', '13', '13', '21', '5', '15', '27', '28']
ACC_train: 0.6
ClassCL: tensor(0.0969, device='cuda:3')
ClassCL_s1: tensor(0.1

cluster_output: ['8', '10', '13', '26', '5', '26', '31', '7', '0', '28']
labels: ['8', '10', '13', '26', '5', '26', '29', '7', '0', '14']
ACC_train: 0.8
ClassCL: tensor(0.0884, device='cuda:3')
ClassCL_s1: tensor(0.0563, device='cuda:3')
ClassCL_s2: tensor(2.7715, device='cuda:3')
************54************
cluster_output: ['23', '27', '24', '7', '20', '26', '2', '15', '30', '16']
labels: ['23', '27', '24', '13', '25', '12', '2', '15', '30', '16']
ACC_train: 0.7
ClassCL: tensor(0.0895, device='cuda:3')
ClassCL_s1: tensor(0.0628, device='cuda:3')
ClassCL_s2: tensor(2.8004, device='cuda:3')
************55************
cluster_output: ['26', '15', '4', '15', '6', '10', '15', '26', '2', '20']
labels: ['12', '3', '4', '15', '6', '10', '15', '26', '2', '0']
ACC_train: 0.7
ClassCL: tensor(0.0888, device='cuda:3')
ClassCL_s1: tensor(0.0719, device='cuda:3')
ClassCL_s2: tensor(2.7707, device='cuda:3')
************56************
cluster_output: ['4', '21', '30', '11', '12', '17', '1', '21', '1', 

...

cluster_output: ['3', '23', '20', '14', '3', '10', '20', '7', '26', '1']
labels: ['3', '23', '20', '14', '3', '10', '20', '7', '26', '1']
ACC_train: 1.0
ClassCL: tensor(0.0788, device='cuda:3')
ClassCL_s1: tensor(0.0019, device='cuda:3')
ClassCL_s2: tensor(2.5184, device='cuda:3')
************2892************
cluster_output: ['28', '26', '28', '19', '20', '30', '1', '1', '25', '10']
labels: ['28', '26', '28', '19', '20', '30', '1', '28', '25', '10']
ACC_train: 0.9
ClassCL: tensor(0.0831, device='cuda:3')
ClassCL_s1: tensor(0.0407, device='cuda:3')
ClassCL_s2: tensor(2.6181, device='cuda:3')
************2893************
cluster_output: ['18', '29', '4', '16', '14', '2', '0', '8', '26', '18']
labels: ['18', '29', '4', '16', '14', '2', '0', '8', '26', '18']
ACC_train: 1.0
ClassCL: tensor(0.0787, device='cuda:3')
ClassCL_s1: tensor(0.0005, device='cuda:3')
ClassCL_s2: tensor(2.5180, device='cuda:3')
************2894************
cluster_output: ['29', '5', '27', '20', '2', '1', '31', '24', 

ClassCL: tensor(0.0792, device='cuda:3')
ClassCL_s1: tensor(0.0068, device='cuda:3')
ClassCL_s2: tensor(2.5279, device='cuda:3')
************2918************
cluster_output: ['1', '12', '11', '26', '23', '11', '30', '20', '18', '1']
labels: ['1', '26', '11', '26', '23', '11', '30', '20', '18', '1']
ACC_train: 0.9
ClassCL: tensor(0.0825, device='cuda:3')
ClassCL_s1: tensor(0.0192, device='cuda:3')
ClassCL_s2: tensor(2.6200, device='cuda:3')
************2919************
cluster_output: ['16', '6', '20', '0', '24', '18', '13', '11', '29', '23']
labels: ['16', '6', '20', '0', '24', '18', '13', '11', '29', '23']
ACC_train: 1.0
ClassCL: tensor(0.0788, device='cuda:3')
ClassCL_s1: tensor(0.0023, device='cuda:3')
ClassCL_s2: tensor(2.5180, device='cuda:3')
************2920************
cluster_output: ['11', '30', '18', '13', '22', '0', '10', '18', '28', '14']
labels: ['11', '30', '18', '13', '22', '0', '10', '18', '28', '14']
ACC_train: 1.0
ClassCL: tensor(0.0787, device='cuda:3')
ClassCL_s1: 

cluster_output: ['29', '21', '22', '11', '24', '19', '17', '18', '30', '23']
labels: ['29', '21', '22', '11', '24', '25', '17', '18', '30', '23']
ACC_train: 0.9
ClassCL: tensor(0.0828, device='cuda:3')
ClassCL_s1: tensor(0.0313, device='cuda:3')
ClassCL_s2: tensor(2.6174, device='cuda:3')
************2945************
cluster_output: ['18', '15', '15', '4', '27', '29', '16', '25', '18', '1']
labels: ['18', '15', '15', '4', '27', '29', '16', '25', '18', '1']
ACC_train: 1.0
ClassCL: tensor(0.0787, device='cuda:3')
ClassCL_s1: tensor(0.0002, device='cuda:3')
ClassCL_s2: tensor(2.5180, device='cuda:3')
************2946************
cluster_output: ['18', '10', '10', '21', '19', '30', '4', '28', '27', '27']
labels: ['18', '10', '10', '21', '19', '30', '4', '28', '27', '27']
ACC_train: 1.0
ClassCL: tensor(0.0788, device='cuda:3')
ClassCL_s1: tensor(0.0013, device='cuda:3')
ClassCL_s2: tensor(2.5206, device='cuda:3')
************2947************
cluster_output: ['10', '30', '15', '8', '7', '26'

ClassCL: tensor(0.0797, device='cuda:3')
ClassCL_s1: tensor(0.0152, device='cuda:3')
ClassCL_s2: tensor(2.5340, device='cuda:3')
************2971************
cluster_output: ['17', '31', '11', '29', '2', '29', '18', '21', '16', '13']
labels: ['17', '31', '11', '29', '2', '29', '18', '21', '16', '13']
ACC_train: 1.0
ClassCL: tensor(0.0792, device='cuda:3')
ClassCL_s1: tensor(0.0125, device='cuda:3')
ClassCL_s2: tensor(2.5225, device='cuda:3')
************2972************
cluster_output: ['17', '14', '1', '7', '16', '8', '13', '19', '4', '20']
labels: ['17', '14', '1', '7', '16', '8', '13', '19', '4', '20']
ACC_train: 1.0
ClassCL: tensor(0.0789, device='cuda:3')
ClassCL_s1: tensor(0.0032, device='cuda:3')
ClassCL_s2: tensor(2.5213, device='cuda:3')
************2973************
cluster_output: ['21', '1', '4', '8', '2', '10', '3', '18', '23', '19']
labels: ['21', '1', '4', '8', '2', '10', '3', '18', '23', '19']
ACC_train: 1.0
ClassCL: tensor(0.0789, device='cuda:3')
ClassCL_s1: tensor(0.0

cluster_output: ['19', '15', '19', '27', '4', '10', '4', '8', '22', '0']
labels: ['19', '15', '19', '27', '4', '10', '4', '8', '22', '0']
ACC_train: 1.0
ClassCL: tensor(0.0787, device='cuda:3')
ClassCL_s1: tensor(0.0010, device='cuda:3')
ClassCL_s2: tensor(2.5181, device='cuda:3')
************2998************
cluster_output: ['13', '20', '13', '13', '3', '25', '20', '24', '18', '21']
labels: ['13', '20', '13', '13', '3', '1', '20', '24', '18', '21']
ACC_train: 0.9
ClassCL: tensor(0.0814, device='cuda:3')
ClassCL_s1: tensor(0.0134, device='cuda:3')
ClassCL_s2: tensor(2.5903, device='cuda:3')
************2999************
cluster_output: ['30', '31', '15', '13', '2', '25', '9', '11', '7', '2']
labels: ['4', '31', '15', '13', '2', '25', '9', '11', '7', '2']
ACC_train: 0.9
ClassCL: tensor(0.0820, device='cuda:3')
ClassCL_s1: tensor(0.0158, device='cuda:3')
ClassCL_s2: tensor(2.6091, device='cuda:3')


## 拯救All test计划

In [10]:
pwd

'/home/comp/20483007/QY/EMNLP2023'

In [11]:
#### Test_All
All_path = '/home/comp/20483007/QY/EMNLP2023/Dataset/Processed/HWU/Test_all.txt'
All_x = []
All_y = []
for line in open(All_path):
    All_y.append(line.split('\t')[1][:-1].replace('_',' '))
    All_x.append(line.split('\t')[0])

Test_base_x = []
Test_base_y = []
Test_base_y_id = []
for i in range(len(All_y)):
    if All_y[i] in label_yd_yi_dic.keys():
        Test_base_x.append(All_x[i])
        Test_base_y.append(All_y[i])
        Test_base_y_id.append(label_yd_yi_dic[All_y[i]])
print('Test_base_y_id:',len(Test_base_y_id))

print("Cv_x:",Test_base_x[:5])
print("***********************")
print("Test_base_y:",Test_base_y[:5])
print("***********************")
print("Test_base_y_id:",Test_base_y_id[:5])

Test_base_y_id: 548
Cv_x: ['disable the alarm which is set at nine thirty pm', 'delete scheduled alarm', 'turn off my first alarm', 'please cancel all alarms for tomorrow', 'please delete the wednesday evening alarm']
***********************
Test_base_y: ['alarm remove', 'alarm remove', 'alarm remove', 'alarm remove', 'alarm remove']
***********************
Test_base_y_id: ['15', '15', '15', '15', '15']


In [13]:
val_dataset = AugmentPairSamples(Test_base_x, Test_base_y_id, Test_base_y)
val_loader = util_data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

ckpt = '../EMNLP2023/Result/PreModel_HWU.pt'
model = torch.load(ckpt)

In [14]:
label = []
pre_result = []
for i in np.arange(54):  ## 25 epochs
    model.training_flag = False       
    print('************'+str(i)+'************')        
    try:
        batch = next(val_loader_iter)
    except:
        val_loader_iter = iter(val_loader)
        batch = next(val_loader_iter)      

    feats, labels = prepare_task_input(batch, is_contrastive=True)        
    _, _, _, labels, cluster_result = model.forward(feats, labels, aggregate=True)
    label += labels
    pre_result += cluster_result


************0************


/home/comp/20483007/miniconda3/envs/DS/lib/python3.7/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


cluster_output: ['10', '3', '22', '11', '15', '27', '30', '9', '9', '26']
labels: ['10', '3', '22', '11', '15', '27', '30', '9', '9', '26']
ACC_train: 1.0
************1************
cluster_output: ['19', '13', '17', '7', '27', '18', '17', '6', '25', '1']
labels: ['19', '13', '17', '7', '27', '18', '17', '6', '23', '1']
ACC_train: 0.9
************2************
cluster_output: ['26', '7', '8', '19', '28', '14', '23', '24', '7', '14']
labels: ['26', '7', '11', '19', '28', '21', '23', '24', '7', '14']
ACC_train: 0.8
************3************
cluster_output: ['2', '2', '10', '10', '7', '12', '24', '5', '16', '3']
labels: ['2', '2', '10', '10', '7', '12', '24', '5', '16', '3']
ACC_train: 1.0
************4************
cluster_output: ['20', '14', '15', '30', '15', '31', '5', '0', '6', '5']
labels: ['20', '14', '15', '30', '15', '31', '5', '0', '6', '5']
ACC_train: 1.0
************5************
cluster_output: ['1', '0', '13', '28', '4', '17', '1', '4', '10', '20']
labels: ['1', '0', '13', '28

cluster_output: ['12', '22', '30', '8', '26', '29', '13', '28', '19', '9']
labels: ['12', '22', '30', '8', '26', '29', '0', '23', '19', '9']
ACC_train: 0.8
************47************
cluster_output: ['24', '5', '14', '1', '1', '14', '24', '21', '13', '27']
labels: ['24', '5', '14', '1', '1', '14', '24', '21', '13', '27']
ACC_train: 1.0
************48************
cluster_output: ['4', '5', '2', '11', '20', '12', '28', '11', '27', '19']
labels: ['4', '5', '2', '11', '20', '12', '28', '25', '27', '19']
ACC_train: 0.9
************49************
cluster_output: ['14', '2', '9', '8', '6', '3', '0', '2', '22', '0']
labels: ['14', '2', '9', '8', '6', '20', '0', '2', '22', '0']
ACC_train: 0.9
************50************
cluster_output: ['25', '26', '10', '7', '17', '30', '9', '12', '25', '30']
labels: ['25', '26', '10', '7', '17', '30', '9', '12', '25', '30']
ACC_train: 1.0
************51************
cluster_output: ['17', '12', '13', '7', '14', '4', '20', '26', '3', '26']
labels: ['17', '12', '

In [16]:
print("ACC:",accuracy_score(label,pre_result))
print("F1:",f1_score(label,pre_result, average='macro'))

ACC: 0.9259259259259259
F1: 0.9239530123806239


### 验证过程

In [12]:
CVbase_path = '../EMNLP2023/Dataset/Processed/HWU/Pretrain.txt'
CVbase_list = []
for line in open(CVbase_path): 
    CVbase_list.append(line[:-1])

Cv_x = []
Cv_y = []
for line in CVbase_list:
    if len(line.split('\t')) != 2:
        continue
    else:
        Cv_y.append(line.split('\t')[-1].replace('_',' '))
        Cv_x.append(line.split('\t')[0])
print(Cv_y[:5])   

pre_yd_yi_dic = label_yd_yi_dic
Cv_y_id = [pre_yd_yi_dic[Cv_y[i]] for i in range(len(Cv_x))]
print(len(Cv_y_id))
print(len(list(set(Cv_y_id))))
# print("Cv_x:",Cv_x)
# print("***********************")
# print("Cv_y:",Cv_y)
# print("***********************")
# print("Cv_y_id:",Cv_y_id)

['alarm remove', 'alarm remove', 'alarm remove', 'alarm remove', 'alarm remove']
4541
32


In [13]:
val_dataset = AugmentPairSamples(Cv_x, Cv_y_id, Cv_y)
val_loader = util_data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

In [14]:
ckpt = '../EMNLP2023/Result/PreModel_HWU.pt'
label = []
pre_result = []
for i in np.arange(152):  ## 25 epochs
    model.training_flag = False       
    print('************'+str(i)+'************')        
    try:
        batch = next(val_loader_iter)
    except:
        val_loader_iter = iter(val_loader)
        batch = next(val_loader_iter)      

    feats, labels = prepare_task_input(batch, is_contrastive=True)        
    _, _, _, labels, cluster_result = model.forward(feats, labels, aggregate=True)
    label += labels
    pre_result += cluster_result


************0************
cluster_output: ['24', '3', '9', '10', '5', '7', '5', '18', '27', '9']
labels: ['24', '3', '9', '10', '5', '7', '5', '18', '27', '9']
ACC_train: 1.0
************1************
cluster_output: ['17', '31', '13', '21', '22', '10', '0', '29', '21', '22']
labels: ['17', '31', '13', '21', '22', '10', '0', '29', '21', '22']
ACC_train: 1.0
************2************
cluster_output: ['12', '13', '10', '26', '30', '2', '12', '13', '25', '1']
labels: ['12', '13', '10', '26', '30', '2', '12', '13', '25', '1']
ACC_train: 1.0
************3************
cluster_output: ['22', '8', '28', '0', '29', '0', '0', '20', '0', '30']
labels: ['22', '8', '28', '0', '29', '0', '0', '20', '0', '30']
ACC_train: 1.0
************4************
cluster_output: ['23', '31', '3', '19', '28', '26', '20', '2', '31', '0']
labels: ['23', '31', '3', '19', '28', '26', '20', '2', '31', '0']
ACC_train: 1.0
************5************
cluster_output: ['29', '19', '27', '18', '12', '2', '24', '27', '1', '13'

cluster_output: ['8', '18', '4', '31', '8', '22', '15', '12', '21', '27']
labels: ['8', '18', '4', '31', '8', '22', '15', '12', '21', '27']
ACC_train: 1.0
************46************
cluster_output: ['18', '6', '27', '19', '11', '29', '21', '17', '27', '2']
labels: ['18', '6', '27', '19', '11', '29', '21', '17', '27', '2']
ACC_train: 1.0
************47************
cluster_output: ['3', '13', '15', '23', '5', '31', '15', '13', '10', '10']
labels: ['3', '13', '15', '23', '5', '31', '15', '13', '10', '10']
ACC_train: 1.0
************48************
cluster_output: ['3', '4', '24', '10', '3', '25', '24', '14', '15', '16']
labels: ['3', '4', '24', '10', '3', '25', '24', '14', '15', '16']
ACC_train: 1.0
************49************
cluster_output: ['29', '29', '5', '4', '0', '17', '23', '2', '8', '1']
labels: ['29', '29', '5', '4', '0', '17', '23', '2', '8', '1']
ACC_train: 1.0
************50************
cluster_output: ['18', '2', '30', '5', '6', '26', '4', '9', '4', '4']
labels: ['18', '2', '3

cluster_output: ['20', '8', '5', '20', '4', '15', '10', '14', '3', '10']
labels: ['20', '8', '5', '20', '4', '15', '10', '14', '3', '10']
ACC_train: 1.0
************92************
cluster_output: ['31', '12', '17', '19', '18', '8', '10', '21', '9', '14']
labels: ['31', '12', '17', '19', '18', '8', '10', '21', '9', '14']
ACC_train: 1.0
************93************
cluster_output: ['6', '18', '9', '8', '5', '3', '20', '13', '21', '5']
labels: ['6', '18', '9', '8', '5', '3', '20', '13', '21', '5']
ACC_train: 1.0
************94************
cluster_output: ['30', '3', '3', '31', '21', '6', '8', '30', '2', '24']
labels: ['30', '3', '3', '31', '21', '6', '8', '30', '2', '24']
ACC_train: 1.0
************95************
cluster_output: ['25', '18', '19', '23', '2', '25', '31', '21', '19', '27']
labels: ['25', '18', '19', '23', '2', '25', '31', '21', '19', '27']
ACC_train: 1.0
************96************
cluster_output: ['31', '10', '16', '14', '11', '23', '2', '20', '1', '30']
labels: ['31', '19', 

cluster_output: ['13', '2', '13', '6', '31', '19', '18', '30', '1', '11']
labels: ['13', '2', '13', '6', '31', '19', '18', '30', '1', '11']
ACC_train: 1.0
************137************
cluster_output: ['25', '25', '17', '20', '0', '30', '27', '20', '21', '0']
labels: ['25', '25', '17', '20', '0', '30', '27', '20', '21', '0']
ACC_train: 1.0
************138************
cluster_output: ['19', '25', '16', '31', '2', '26', '16', '3', '9', '5']
labels: ['19', '25', '16', '31', '2', '26', '16', '3', '9', '5']
ACC_train: 1.0
************139************
cluster_output: ['15', '9', '15', '20', '6', '22', '24', '27', '21', '4']
labels: ['15', '9', '15', '20', '6', '22', '24', '27', '21', '4']
ACC_train: 1.0
************140************
cluster_output: ['4', '10', '2', '10', '3', '8', '0', '11', '0', '13']
labels: ['4', '10', '2', '10', '3', '8', '0', '11', '0', '13']
ACC_train: 1.0
************141************
cluster_output: ['22', '31', '3', '6', '31', '5', '21', '3', '23', '7']
labels: ['22', '31'

In [15]:
ACC = accuracy_score(label,pre_result)
print("ACC_train:",ACC)

ACC_train: 0.9259259259259259


In [ ]:
# ACC_train: 0.99375